In [1]:
# imports

import csv
import json
import os
import re
from pprint import pprint
import time
import re
import unicodedata
import openai
import os
import sys
from contextlib import contextmanager
from copy import deepcopy
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import SpectralEmbedding
import pickle
import sknetwork as skn
from random import *

import matplotlib.pyplot as plt
import seaborn as sns


import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models
import PyPDF2

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

import nltk
nltk.download(["names", "stopwords", "state_union", "twitter_samples", "movie_reviews", "averaged_perceptron_tagger", "vader_lexicon", "punkt", "wordnet"])
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
ps = nltk.porter.PorterStemmer()

from gsdmm import MovieGroupProcess

from sentence_transformers import SentenceTransformer

import spacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

from itertools import combinations

from collections import Counter
# set seed for reproducibility
# np.random.seed(493)

C:\Users\Seth Benson\AppData\Roaming\Python\Python310\site-packages\PyPDF2\__init__.py:21: DeprecationWarning: PyPDF2 is deprecated. Please move to the pypdf library instead.
  warnings.warn(
[nltk_data] Downloading package names to C:\Users\Seth
[nltk_data]     Benson\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Seth
[nltk_data]     Benson\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to C:\Users\Seth
[nltk_data]     Benson\AppData\Roaming\nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to C:\Users\Seth
[nltk_data]     Benson\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to C:\Users\Seth
[nltk_data]     Benson\AppData\Roaming\nltk_data...
[nltk_data]   Package

In [2]:
#text cleaning functions

def tokeniz(sentences):
    for sentence in sentences:
        yield (gensim.utils.simple_preprocess(str(sentence), deacc=True))


# function to filter out stopwords
def remove_stopwords(texts):
    return [
        [word for word in simple_preprocess(str(doc)) if word not in stop_words]
        for doc in texts
    ]


# function for lemmatization
def lemmatize(texts, allowed_postags=["NOUN", "ADJ", "VERB"]):
    texts_op = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_op.append(
            [token.lemma_ for token in doc if token.pos_ in allowed_postags]
        )
    return texts_op


def top_words(cluster_word_distribution, top_cluster, values):
    words = []
    for cluster in top_cluster:
        sort_dicts = sorted(
            cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True
        )[:values]
        words.append([w[0] for w in sort_dicts])
        # print('Cluster %s : %s'%(cluster,sort_dicts))
        # print('-'*120)
    words1 = deepcopy(words)
    for i in range(len(words)):
        for j in range(len(words[i])):
            for k in range(len(words)):
                if k != i and words[i][j] in words[k]:
                    try:
                        words1[i].remove(words[i][j])
                        words1[k].remove(words[i][j])
                    except:
                        pass
    return words1


def create_topics_dataframe(data_text, mgp, threshold, topic_dict, stem_text):
    result = pd.DataFrame(columns=["text", "topic", "stems"])
    for i, text in enumerate(data_text):
        result.at[i, "text"] = text
        result.at[i, "stems"] = stem_text[i]
        prob = mgp.choose_best_label(stem_text[i])
        if prob[1] >= threshold:
            result.at[i, "topic"] = topic_dict[prob[0]]
        else:
            result.at[i, "topic"] = "Other"
    return result

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout


# Building Bigram & Trigram Models
#bigram = gensim.models.Phrases(processed_data, min_count=5, threshold=100)
#trigram = gensim.models.Phrases(bigram[processed_data], threshold=100)
#bigram_mod = gensim.models.phrases.Phraser(bigram)
#trigram_mod = gensim.models.phrases.Phraser(trigram)

# function to create bigrams
def create_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


# function to create trigrams
def create_trigrams(texts):
    [trigram_mod[bigram_mod[doc]] for doc in texts]

In [ ]:
def top_words(cluster_word_distribution, top_cluster, values):
    words = []
    for cluster in top_cluster:
        sort_dicts = sorted(
            cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True
        )[:values]
        words.append([w[0] for w in sort_dicts])
        # print('Cluster %s : %s'%(cluster,sort_dicts))
        # print('-'*120)
    return words1

In [2]:
sid = SentimentIntensityAnalyzer()
nlp = spacy.load("en_core_web_sm")
NER = spacy.load("en_core_web_sm")
raw_text="The Indian Space Research Organisation or DOS is the national space agency of India, headquartered in Bengaluru. It operates under Department of Space which is directly overseen by the Prime Minister of India while Chairman of ISRO acts as executive of DOS as well."
text1= NER(raw_text)
words = []
for word in text1.ents:
    words.append(word.text)
words = Counter(words)
print(words)

Counter({'DOS': 2, 'India': 2, 'The Indian Space Research Organisation': 1, 'Bengaluru': 1, 'Department of Space': 1, 'ISRO': 1})


In [25]:
def transcript_cleaning(month):
    tr_tp_st = [[], [], [], []]

    gl = pd.read_csv("foxGuestList.csv", encoding="windows-1252")
    fox = open("fox_text.txt", "r", encoding="windows-1252")
    fox = fox.readlines()
    for l in range(len(gl)):
        #######################################
        ##Turning transcript into df of lines##
        #######################################
        lines = pd.DataFrame(
            columns=["Date", "Start_Hour", "Program", "Title", "Speaker", "Line"]
        )
        # print(lines)
        transcript = fox[l]
        meta = gl.iloc[l, :]
        dt = meta["Date"]
        if month in str(dt):
            start = time.time()
            if randint(1, 100) <= 50:
                print(l)
            chunks = transcript.split("|")
            line = ""
            spkr = ""
            for c in chunks:
                if (
                    "Page" not in c[0:10]
                    and "...." not in c
                    and c[0:15] not in meta["Title"]
                ):
                    # print(c)
                    if ":" in c:
                        c = c.split(":")
                        if str.isupper(c[0]):
                            lines.loc[len(lines)] = [
                                meta["Date"],
                                meta["Start_Hour"],
                                meta["Program"],
                                meta["Title"],
                                spkr,
                                line,
                            ]
                            # print(spkr + ':' + line)
                            spkr = c[0]
                            line = c[1]
                    else:
                        line += c
            lines.loc[len(lines)] = [
                meta["Date"],
                meta["Start_Hour"],
                meta["Program"],
                meta["Title"],
                spkr,
                line,
            ]
            ckp1 = time.time()
            print(f"Checkpoint 1: {start-ckp1}")
            #########################
            ##Keyword and Sentiment##
            #########################

            #lines['Sentence'] = lines['Line'].apply(lambda x: [str(sent) for sent in nlp(x).sents])
            #print(lines['Sentence'][1])
            #sentences = lines.explode('Sentence')
            #print(sentences['Sentence'][0:5])
            #sentences = sentences.dropna(subset = ['Sentence'])
            lines['Line'] = lines['Line'].apply(lambda x: x.lower())
            lines["VADER"] = lines["Line"].apply(
                lambda x: list(sid.polarity_scores(x).values())[3]
            )
            ckp2 = time.time()
            print(f"Checkpoint 2: {start-ckp2}")

            word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            tokens = nltk.word_tokenize(raw_text)
            words = Counter(tokens).most_common()
            print(words[0:10])
            words1 = []
            i = 0
            while len(words1) < 15 and i < len(words):
                w = words[i][0]
                #print(w)
                tag =  nltk.pos_tag([w])
                t = tag[0][1] 
                if t in ['NN','NNS','NNP','NNPS','FW','SYM'] and w not in ['video','clip','i','end','begin']: #or t == "FW":
                    words1.append(w)
                i += 1
            #for word in text1.ents:
            #    if word.label_ not in ['TIME','MONEY','PERCENT','DATE','QUANTITY','ORDINAL','CARDINAL']:
            #        words.append(word.text)
            #words = Counter(words).most_common()
            ##print(words)
            #words = words[0:15]
            ckp3 = time.time()
            print(f"Checkpoint 3: {start-ckp3}")
            for w in words1:
                try:
                    w1 = w
                    wdf = lines[lines['Line'].str.contains(w1)]
                    w_list.append(w1)
                    sent_list = list(wdf["VADER"])
                    word_sentiment_list.append(sent_list)
                except:
                    pass
            ckp4 = time.time()
            print(f"Checkpoint 4: {start-ckp4}")
            # print(w_list)
                

            ######################
            ##Storing Each Topic##
            ######################
            for w in range(len(w_list)):
                if len(word_sentiment_list[w]) > 2:
                    tr_tp_st[0].append(l)
                    tr_tp_st[1].append(w_list[w])
                    tr_tp_st[2].append(np.mean(word_sentiment_list[w]))
                    tr_tp_st[3].append(lines["Program"][0])
                    print(str(l) +'   ' +  w_list[w] + '    ' + str(np.mean(word_sentiment_list[w])) + '   ' + lines["Program"][0])
            ckp5 = time.time()
            print(f"Checkpoint 5: {start-ckp5}")

    #print(tr_tp_st)
    return tr_tp_st

In [26]:
# Save out processed and topic modeled data
months = ['January','February','March','April']
for m in months:
    tr_tp_st = transcript_cleaning(m)
    tdf = pd.DataFrame(zip(tr_tp_st[0], tr_tp_st[1], tr_tp_st[2], tr_tp_st[3]), 
                       columns=["Transcript", "Topic", "Sentiment", "Program"],)
    with open('processed_transcripts_' + m + '2020.pkl', 'wb') as file:
        pickle.dump(tr_tp_st, file)
    tdf.to_csv('processed_transcripts_' + m + '2020.csv')

Checkpoint 1: -0.2480027675628662
Checkpoint 2: -0.35145115852355957
[(',', 470), ('the', 400), ('.', 363), ('to', 188), ('of', 186), ('and', 182), ('that', 131), ('a', 123), ('in', 100), ('they', 100)]
Checkpoint 3: -0.47641944885253906
Checkpoint 4: -0.4874241352081299
21   trump    -0.072385   Fox News Network INGRAHAM ANGLE
21   comey    -0.011350000000000013   Fox News Network INGRAHAM ANGLE
21   democrats    -0.11733157894736841   Fox News Network INGRAHAM ANGLE
21   president    -0.05057142857142859   Fox News Network INGRAHAM ANGLE
21   think    -0.045255   Fox News Network INGRAHAM ANGLE
21   way    0.3090545454545454   Fox News Network INGRAHAM ANGLE
21   fbi    0.2534625   Fox News Network INGRAHAM ANGLE
21   impeachment    -0.062045454545454536   Fox News Network INGRAHAM ANGLE
21   mueller    -0.1347142857142857   Fox News Network INGRAHAM ANGLE
21   report    -0.06666153846153847   Fox News Network INGRAHAM ANGLE
21   generation    0.3453857142857143   Fox News Network IN

KeyboardInterrupt: 